<a href="https://colab.research.google.com/github/LeoFernanndes/portfolio/blob/master/MachineFailuresModelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Failures Modelling 

Seguindo a análise exploratória de dados disponível em: https://github.com/LeoFernanndes/portfolio/blob/master/MachineFailuresEDA.ipynb, passamos à modelagem preditiva de falhas.

Pontos relevantes enontrados na EDA: de todas as variáveis apresentadas no problema, nos parece plausível que ao menso 90% da explicabilidade da ocorrência de falhas em operação sejam suportados por temperatura, humidade e tempo decorrido desde a última falha.

Resumo: Passando por um modelo baseline linear pra checar a dúvida levantada na primeira parte o trabalho, seguido de um modelo baseline de verdade, baseado em árvores e 3 opçãoes de melhoria nos resultados, percebemos que não há grandes avanços com relação ao baseline baseado em árvores. Nossa ideia inicial de aumentar a confiabilidade foi satisfeita observando medias e desvios padrão das previsões em 5 splits randomizadas pelo mesmo seed. Fica um pouco complexo avaliar qual técnica de otimização trouxe melhores resultados. Seria necessaria uma investigação de base estatística um pouco mais avançada para ponderar os efeitos de aleatoriedade sobre a diferença entre featre selection e ajuste de hiperparâmetros por exemplo, mas saímos com uma f1 score que gira em torno de 0.82 para um problema extremamente desbalanceado o que nos parece uma solução de primeira ordem suficientemente interessante.

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


raw_data_url = "https://raw.githubusercontent.com/LeoFernanndes/datasets/master/dataset-machine_failures.csv"
raw_data = pd.read_csv(raw_data_url)
raw_data

,Date,Temperature,Humidity,Operator,Measure1,Measure2,Measure3,Measure4,Measure5,Measure6,Measure7,Measure8,Measure9,Measure10,Measure11,Measure12,Measure13,Measure14,Measure15,Hours Since Previous Failure,Failure,﻿Date.year,﻿Date.month,﻿Date.day-of-month,﻿Date.day-of-week,﻿Date.hour,﻿Date.minute,﻿Date.second
0,2016-01-01 00:00:00,67,82,Operator1,291,1,1,1041,846,334,706,1086,256,1295,766,968,1185,1355,1842,90,No,2016,1,1,5,0,0,0
1,2016-01-01 01:00:00,68,77,Operator1,1180,1,1,1915,1194,637,1093,524,919,245,403,723,1446,719,748,91,No,2016,1,1,5,1,0,0
2,2016-01-01 02:00:00,64,76,Operator1,1406,1,1,511,1577,1121,1948,1882,1301,273,1927,1123,717,1518,1689,92,No,2016,1,1,5,2,0,0
3,2016-01-01 03:00:00,63,80,Operator1,550,1,1,1754,1834,1413,1151,945,1312,1494,1755,1434,502,1336,711,93,No,2016,1,1,5,3,0,0
4,2016-01-01 04:00:00,65,81,Operator1,1928,1,2,1326,1082,233,1441,1736,1033,1549,802,1819,1616,1507,507,94,No,2016,1,1,5,4,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8779,2016-12-31 19:00:00,66,79,Operator8,937,0,2,1875,1719,219,748,1901,819,546,901,1512,1477,537,410,7,No,2016,12,31,6,19,0,0
8780,2016-12-31 20:00:00,60,77,Operator8,379,3,0,814,1990,1606,1346,962,875,1067,608,251,1476,522,943,8,No,2016,12,31,6,20,0,0
8781,2016-12-31 21:00:00,61,77,Operator8,595,3,1,1659,1134,1314,907,1119,1623,272,1149,1951,1835,1554,200,9,No,2016,12,31,6,21,0,0
8782,2016-12-31 22:00:00,72,81,Operator8,1038,3,2,254,1400,564,216,1011,1909,502,470,1331,1696,229,1192,10,No,2016,12,31,6,22,0,0


## Baseline model

### Preprocessing

In [2]:
raw_data.dtypes

Date                            object
Temperature                      int64
Humidity                         int64
Operator                        object
Measure1                         int64
Measure2                         int64
Measure3                         int64
Measure4                         int64
Measure5                         int64
Measure6                         int64
Measure7                         int64
Measure8                         int64
Measure9                         int64
Measure10                        int64
Measure11                        int64
Measure12                        int64
Measure13                        int64
Measure14                        int64
Measure15                        int64
Hours Since Previous Failure     int64
Failure                         object
﻿Date.year                       int64
﻿Date.month                      int64
﻿Date.day-of-month               int64
﻿Date.day-of-week                int64
﻿Date.hour               

In [3]:
df = raw_data.copy()

df['Date'] = pd.to_datetime(df['Date'])
df['Failure'] = df['Failure'].map({"Yes": 1, "No": 0})
proporcao_falha_operador = df.groupby(by='Operator').mean()['Failure']
df['Operator'] = df['Operator'].map(proporcao_falha_operador)

Transformações básicas neessárias visto que a maioria dos modelos do scikit-learn necessitam de features numericas. 

Pro caso de operador, vários encodings são válidos, mas no nosso caso usamos o target encoding por proporçao de falhas para que a feature passe a assumir algum poder preditivo além de simplesmente ser transformada em númreo aleatorios.


### Baseline model
Baseline é o modelo mais barato a ser desenvolvido. Um MVP.

In [4]:
df.columns

Index(['Date', 'Temperature', 'Humidity', 'Operator', 'Measure1', 'Measure2',
       'Measure3', 'Measure4', 'Measure5', 'Measure6', 'Measure7', 'Measure8',
       'Measure9', 'Measure10', 'Measure11', 'Measure12', 'Measure13',
       'Measure14', 'Measure15', 'Hours Since Previous Failure', 'Failure',
       '﻿Date.year', '﻿Date.month', '﻿Date.day-of-month', '﻿Date.day-of-week',
       '﻿Date.hour', '﻿Date.minute', '﻿Date.second'],
      dtype='object')

In [5]:
import random
model_ramdomness_seed = random.randint(1, 1000)
model_ramdomness_seed

439

Essa etapa é imortante para que possamos usar a aleatoriedade a nosso favor, mas de modo que em uma mesma rodada, todos os modelos sejam poderados por um mesmo algoritmo de divisão em treino e teste para que possamos saber qual se comporta melhor

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, StratifiedKFold


x_features =['Temperature', 'Humidity', 'Operator', 'Measure1', 'Measure2',
       'Measure3', 'Measure4', 'Measure5', 'Measure6', 'Measure7', 'Measure8',
       'Measure9', 'Measure10', 'Measure11', 'Measure12', 'Measure13',
       'Measure14', 'Measure15', 'Hours Since Previous Failure',
       '﻿Date.year', '﻿Date.month', '﻿Date.day-of-month', '﻿Date.day-of-week',
       '﻿Date.hour', '﻿Date.minute', '﻿Date.second']
y_feature = 'Failure'


model = LogisticRegression(max_iter=10000)
folds = StratifiedKFold(shuffle=True, random_state=model_ramdomness_seed)

result = cross_val_score(model, df[x_features], df[y_feature], scoring="f1", cv=folds)
print(f"{result.mean()} ± {2 * result.std()}")

0.7475023041474654 ± 0.36857144654827256


O modelo apresentado não necessariamente o mais barato como anteriormente porque por padrão, modelos lineares, para uma mesma quantidade detempo gasto no desenvolvimento, têm performace inferior a modelos baseados em árvores como veremos a segir.

In [7]:
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold


x_features =['Temperature', 'Humidity', 'Operator', 'Measure1', 'Measure2',
       'Measure3', 'Measure4', 'Measure5', 'Measure6', 'Measure7', 'Measure8',
       'Measure9', 'Measure10', 'Measure11', 'Measure12', 'Measure13',
       'Measure14', 'Measure15', 'Hours Since Previous Failure',
       '﻿Date.year', '﻿Date.month', '﻿Date.day-of-month', '﻿Date.day-of-week',
       '﻿Date.hour', '﻿Date.minute', '﻿Date.second']
y_feature = 'Failure'


model = XGBClassifier()
folds = StratifiedKFold(shuffle=True, random_state=model_ramdomness_seed)

result = cross_val_score(model, df[x_features], df[y_feature], scoring="f1", cv=folds)
print(f"{result.mean()} ± {2 * result.std()}")

0.8196451612903226 ± 0.21223847127705658


Na media, o xgboost apresenta um resultado melhor, mas olhando com cuidado isso não nos parece uma vitoria. Um bom modelo precisa transmitir a confiança de sempre prever com a mesma acurácia e para isso é importante que não só a métrica de intersse, no caso o f1 score seja a maior possível, mas também que a variância das previsôes em conjuntos de treino e teste diferentes seja a menor possível.

### Aprofundando o XGBoost

Como vimos acima, precisamos tornar nosso modelo mais confiável. Para tal vamos abordar 3 perspetivas da modelagem: normalização de dados, seleção de variáveis e ajuste de hiperparâmetros.

#### Scaling

In [8]:
from sklearn.preprocessing import StandardScaler
import numpy as np


ss = StandardScaler()

scaled_df = df.copy()
for feature in scaled_df.columns:
    if feature == "Failure":
        continue
    scaled_df[feature] = ss.fit_transform(np.array(scaled_df[feature]).reshape(-1, 1))


x_features =['Temperature', 'Humidity', 'Operator', 'Measure1', 'Measure2',
       'Measure3', 'Measure4', 'Measure5', 'Measure6', 'Measure7', 'Measure8',
       'Measure9', 'Measure10', 'Measure11', 'Measure12', 'Measure13',
       'Measure14', 'Measure15', 'Hours Since Previous Failure',
       '﻿Date.year', '﻿Date.month', '﻿Date.day-of-month', '﻿Date.day-of-week',
       '﻿Date.hour', '﻿Date.minute', '﻿Date.second']
y_feature = 'Failure'


model = XGBClassifier()
folds = StratifiedKFold(shuffle=True, random_state=model_ramdomness_seed)

result = cross_val_score(model, scaled_df[x_features], scaled_df[y_feature], scoring="f1", cv=folds)
print(f"{result.mean()} ± {2 * result.std()}")


0.8196451612903226 ± 0.21223847127705658


Nenhuma diferença com relação aos dados não escalados? Pois é. Modelos baseados em árvore acabam não sendo influenciados por transformações monotonicas em suas variáveis, mas podemos tentar algo levemente diferente: do mesmo modo que com o operador, podemos tentar escalar as variáveis pela participação na proporção de falhas

#### Feature Seletion

In [9]:
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold

feature_selection_df = df.copy()
feature_selection_df["random_variable"] = [random.randint(-5, 5) for failure in df['Failure']]
feature_selection_x_features = x_features.copy()
feature_selection_x_features.append("random_variable")

model = XGBClassifier()

model.fit(feature_selection_df[feature_selection_x_features], feature_selection_df[y_feature])
feature_importances = pd.DataFrame({
    'feature': feature_selection_x_features,
    'importance': model.feature_importances_
    }).sort_values(by='importance', ascending=False)

feature_importances

,feature,importance
1,Humidity,0.236069
0,Temperature,0.111014
18,Hours Since Previous Failure,0.052565
26,random_variable,0.048559
15,Measure13,0.043776
13,Measure11,0.041346
16,Measure14,0.036477
23,﻿Date.hour,0.036287
7,Measure5,0.036150
3,Measure1,0.035042


In [10]:
random_variable_importance = feature_importances[feature_importances['feature'] == 'random_variable'].importance.values[0]
important_x_features = feature_importances[feature_importances['importance'] > random_variable_importance]['feature'].values
y_feature = 'Failure'

model = XGBClassifier()
folds = StratifiedKFold(shuffle=True, random_state=model_ramdomness_seed)

result = cross_val_score(model, df[important_x_features], df[y_feature], scoring="f1", cv=folds)
print(f"{result.mean()} ± {2 * result.std()}")

0.8276599326599327 ± 0.17871867379183742


### Hyperparameter tuning

In [17]:
from sklearn.model_selection import GridSearchCV

random_variable_importance = feature_importances[feature_importances['feature'] == 'random_variable'].importance.values[0]
important_x_features = feature_importances[feature_importances['importance'] > random_variable_importance]['feature'].values
y_feature = 'Failure'

model = XGBClassifier()
folds = StratifiedKFold(shuffle=True, random_state=model_ramdomness_seed)

param_grid = {
    "max_depth": [3, 5, len(important_x_features)],
    "learning_rate": [0.01, 0.001],
    "n_estimators": [100, 300, 1000]
    }

search = GridSearchCV(model, param_grid, scoring="f1", cv=folds)
search.fit(df[important_x_features], df[y_feature])


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=439, shuffle=True),
             error_score=nan,
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=3, min_child_weight=1,
                                     missing=None, n_estimators=100, n_jobs=1,
                                     nthread=None, objective='binary:logistic',
                                     random_state=0, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=1, verbosity=1),
             iid='deprecated', n_jobs=None,
             param_grid={'learning_rate': [0.01, 0.001], 'max_depth': [3, 5, 3],
                         

In [18]:
search.best_score_

0.8276599326599327